In [1]:
import os
import numpy as np
import pandas as pd

In [4]:
# Creadint directory to store evaluation metrics for each methods
evaluation_files_loc = '../files/class_balancing_methods_evaluations/'
if not os.path.exists(evaluation_files_loc):
    os.mkdir(evaluation_files_loc)

### Reading data

In [ ]:
# File location of the dataset
data_loc = "./Churn_Modelling.csv"

# Read the CSV file into a Pandas DataFrame, using the first column as the index
df = pd.read_csv(data_loc, index_col=0)

df.head()

### Drop all categorical columns

In [ ]:
df.drop(['CustomerId', 'Surname','Geography', 'Gender'], axis = 1,inplace=True)
df.head()